## Step 1: Load and Inspect Data



In [21]:
import pandas as pd

# Load the data
data_path = "./DataSets/KaggleData/flights.csv"
df_staging = pd.read_csv(data_path)

# Initial inspection
print("Data Sample:")
print(df_staging.head())
print("Data Info:")
print(df_staging.info())
print("Missing Values:")
print(df_staging.isnull().sum())


Data Sample:
   id  year  month  day  dep_time  sched_dep_time  dep_delay  arr_time  \
0   0  2013      1    1     517.0             515        2.0     830.0   
1   1  2013      1    1     533.0             529        4.0     850.0   
2   2  2013      1    1     542.0             540        2.0     923.0   
3   3  2013      1    1     544.0             545       -1.0    1004.0   
4   4  2013      1    1     554.0             600       -6.0     812.0   

   sched_arr_time  arr_delay  ... flight  tailnum origin dest air_time  \
0             819       11.0  ...   1545   N14228    EWR  IAH    227.0   
1             830       20.0  ...   1714   N24211    LGA  IAH    227.0   
2             850       33.0  ...   1141   N619AA    JFK  MIA    160.0   
3            1022      -18.0  ...    725   N804JB    JFK  BQN    183.0   
4             837      -25.0  ...    461   N668DN    LGA  ATL    116.0   

   distance  hour  minute            time_hour                    name  
0      1400     5      1

## Step 2: Data Quality Checks and Data Governance in Staging Area



In [22]:
# Check for duplicates
duplicates = df_staging.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Remove duplicates and handle missing values
df_staging = df_staging.drop_duplicates()
df_staging = df_staging.dropna()  # Or apply other imputation methods as needed

# Ensure data types are consistent and meaningful
for column in df_staging.select_dtypes(include=['object']):
    df_staging[column] = df_staging[column].astype(str).str.strip().str.lower()


Number of duplicate rows: 0


## Step 3: Move Cleaned Data to Bronze Layer



In [23]:
# Saving cleaned data as bronze
df_bronze = df_staging.copy()
bronze_path = "./DataSets/KaggleData/bronze_flights.csv"
df_bronze.to_csv(bronze_path, index=False)
print(f"Bronze data saved at {bronze_path}")


Bronze data saved at ./DataSets/KaggleData/bronze_flights.csv


## Step 4: Data Transformation in Silver Layer
* Apply transformations, e.g., converting timestamps, creating new columns.


In [24]:
# Display column names to verify available columns
print("Available columns in the dataset:")
print(df_staging.columns)


Available columns in the dataset:
Index(['id', 'year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay',
       'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight',
       'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute',
       'time_hour', 'name'],
      dtype='object')


In [25]:
import pandas as pd

# Assuming columns like 'scheduled_arrival' and 'scheduled_departure' might exist as alternatives
df_silver = df_bronze.copy()

# Example transformations, updated for actual columns
if 'arrival_time' in df_silver.columns and 'departure_time' in df_silver.columns:
    df_silver['flight_duration'] = (
        pd.to_datetime(df_silver['arrival_time']) - pd.to_datetime(df_silver['departure_time'])
    ).dt.total_seconds() / 60
elif 'scheduled_arrival' in df_silver.columns and 'scheduled_departure' in df_silver.columns:
    df_silver['flight_duration'] = (
        pd.to_datetime(df_silver['scheduled_arrival']) - pd.to_datetime(df_silver['scheduled_departure'])
    ).dt.total_seconds() / 60
else:
    print("Arrival and departure time columns not found. Please confirm column names.")

# Example to calculate day of the week for the departure date
if 'departure_time' in df_silver.columns:
    df_silver['day_of_week'] = pd.to_datetime(df_silver['departure_time']).dt.dayofweek
elif 'scheduled_departure' in df_silver.columns:
    df_silver['day_of_week'] = pd.to_datetime(df_silver['scheduled_departure']).dt.dayofweek
else:
    print("Departure time column not found. Please confirm column name.")


Arrival and departure time columns not found. Please confirm column names.
Departure time column not found. Please confirm column name.


In [26]:
import pandas as pd
import os

# Define paths for each layer
bronze_path = "./DataSets/KaggleData/flights_bronze.csv"
silver_path = "./DataSets/KaggleData/flights_silver.csv"
gold_path = "./DataSets/KaggleData/flights_gold.csv"

# Ensure the directory exists
os.makedirs("./DataSets/KaggleData/", exist_ok=True)

# Load the initial data into the bronze layer
df_bronze = pd.read_csv('./DataSets/KaggleData/flights.csv')
df_bronze.to_csv(bronze_path, index=False)
print(f"Bronze layer data saved to {bronze_path}")

# Copy data from bronze layer to silver layer
df_silver = df_bronze.copy()

# Check if the relevant columns exist and perform transformations
if 'arr_time' in df_silver.columns and 'dep_time' in df_silver.columns:
    # Calculate flight duration (in minutes)
    df_silver['flight_duration'] = (
        pd.to_datetime(df_silver['arr_time']) - pd.to_datetime(df_silver['dep_time'])
    ).dt.total_seconds() / 60
else:
    print("Arrival ('arr_time') and departure ('dep_time') columns not found. Please confirm column names.")

# Calculate day of the week for the departure time
if 'dep_time' in df_silver.columns:
    df_silver['day_of_week'] = pd.to_datetime(df_silver['dep_time']).dt.dayofweek
else:
    print("Departure time ('dep_time') column not found. Please confirm column name.")

# Check if delay columns exist and calculate them (if they exist)
if 'arr_delay' in df_silver.columns and 'dep_delay' in df_silver.columns:
    # Calculate total delay (in minutes)
    df_silver['total_delay'] = df_silver['arr_delay'] + df_silver['dep_delay']
else:
    print("Arrival delay ('arr_delay') and departure delay ('dep_delay') columns not found. Please confirm column names.")

# Save the transformed data to the silver layer
df_silver.to_csv(silver_path, index=False)
print(f"Silver layer data saved to {silver_path}")

# Additional transformation and data quality checks for gold layer (example: filtering out flights with excessive delays)
df_gold = df_silver[df_silver['total_delay'] <= 120]  # Example: keep flights with <= 2-hour delay
df_gold.to_csv(gold_path, index=False)
print(f"Gold layer data saved to {gold_path}")


Bronze layer data saved to ./DataSets/KaggleData/flights_bronze.csv
Silver layer data saved to ./DataSets/KaggleData/flights_silver.csv
Gold layer data saved to ./DataSets/KaggleData/flights_gold.csv


In [27]:
# df_gold = df_silver.copy()

# # Drop irrelevant or sensitive columns if any
# df_gold = df_gold.drop(columns=['sensitive_column_name'])  # Modify based on data

# # Save to gold layer
# gold_path = "./DataSets/KaggleData/gold_flights.csv"
# df_gold.to_csv(gold_path, index=False)
# print(f"Gold data saved at {gold_path}")


## Step 6: Perform Predictive Analytics (e.g., Flight Delay Prediction)
* Assuming a column "delay" exists, we predict whether a flight will be delayed.

### Predicting Flight Delays Using Random Forest

In this section, we prepare the flight dataset for machine learning by encoding categorical variables such as airline codes and flight routes into numerical values using Label Encoding. This is necessary for training the `RandomForestClassifier` model, which requires numerical input data. After encoding, we split the dataset into training and testing sets and use the trained model to predict flight delays. The performance is evaluated using accuracy and a classification report.


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Load the final gold layer data
df_gold = pd.read_csv('./DataSets/KaggleData/flights_gold.csv')

# Ensure the delay column is created based on arrival and departure delay values
df_gold['delay'] = ((df_gold['arr_delay'] > 0) | (df_gold['dep_delay'] > 0)).astype(int)

# Prepare data by encoding categorical variables
categorical_columns = df_gold.select_dtypes(include=['object']).columns
label_encoders = {}

for column in categorical_columns:
    le = LabelEncoder()
    df_gold[column] = le.fit_transform(df_gold[column].astype(str))
    label_encoders[column] = le  # Store encoders in case of reverse transformation later

# Prepare data for modeling
X = df_gold.drop(columns=['delay', 'arr_delay', 'dep_delay', 'arr_time', 'dep_time'])
y = df_gold['delay']  # Target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9192245915258931
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.96      0.93     47835
           1       0.95      0.87      0.91     42440

    accuracy                           0.92     90275
   macro avg       0.92      0.92      0.92     90275
weighted avg       0.92      0.92      0.92     90275



## Step 7: Bias Detection and Mitigation Using Fairlearn


### 3. Bias Detection with Fairlearn
*We will check for bias based on the 'carrier' (as an example of sensitive attribute)



In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from fairlearn.metrics import demographic_parity_difference
from sklearn.metrics import mean_absolute_error

# Load dataset
data_path = './DataSets/KaggleData/flights_gold.csv'
df = pd.read_csv(data_path)

# Print available columns to verify
print("Available columns in dataset:", df.columns)

# Inspect the unique values in the target column
print("Unique target values in 'dest':", df['dest'].unique())

# Map target to binary values
# Adjust mapping according to your actual target values
binary_mapping = {'EWR': 0, 'SFO': 1}  # Example mapping; adjust as necessary
df['target_binary'] = df['dest'].map(binary_mapping)

# Check for unmapped values (NaN)
if df['target_binary'].isnull().sum() > 0:
    print("Unmapped target values found. Removing them.")
    df = df.dropna(subset=['target_binary'])

# Define features and target
X = df.drop(columns=['id', 'dest', 'target_binary'])  # Adjust feature selection as needed
y = df['target_binary']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encode categorical features (if any)
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

# Define the sensitive feature
sensitive_feature = 'carrier'  # Replace with your sensitive feature column name
X_train_encoded = pd.DataFrame(X_train_encoded)
X_train_encoded[sensitive_feature] = X_train[sensitive_feature].values

# Fit the mitigator using binary labels
mitigator = ExponentiatedGradient(estimator=LinearRegression(),
                                  constraints=DemographicParity())
mitigator.fit(X_train_encoded, y_train, sensitive_features=X_train[sensitive_feature])

# Predict and evaluate
y_pred_mitigated = mitigator.predict(X_test_encoded)

# Assess fairness and performance
dp_diff = demographic_parity_difference(y_test, y_pred_mitigated, sensitive_features=X_test[sensitive_feature])
mae = mean_absolute_error(y_test, y_pred_mitigated)

print("Demographic Parity Difference after mitigation:", dp_diff)
print("Mean Absolute Error after mitigation:", mae)


Available columns in dataset: Index(['id', 'year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay',
       'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight',
       'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute',
       'time_hour', 'name', 'flight_duration', 'day_of_week', 'total_delay'],
      dtype='object')
Unique target values in 'dest': ['IAH' 'MIA' 'BQN' 'ATL' 'ORD' 'FLL' 'IAD' 'MCO' 'PBI' 'TPA' 'LAX' 'SFO'
 'DFW' 'BOS' 'LAS' 'MSP' 'DTW' 'RSW' 'SJU' 'PHX' 'BWI' 'CLT' 'BUF' 'DEN'
 'SNA' 'MSY' 'SLC' 'XNA' 'MKE' 'SEA' 'ROC' 'SYR' 'SRQ' 'RDU' 'CMH' 'JAX'
 'CHS' 'MEM' 'PIT' 'SAN' 'DCA' 'CLE' 'STL' 'MYR' 'JAC' 'MDW' 'HNL' 'BNA'
 'AUS' 'BTV' 'PHL' 'STT' 'EGE' 'AVL' 'PWM' 'IND' 'CAK' 'HOU' 'LGB' 'DAY'
 'ALB' 'BDL' 'MHT' 'MSN' 'GSO' 'CVG' 'GSP' 'GRR' 'MCI' 'ORF' 'SAT' 'PDX'
 'SJC' 'CRW' 'RIC' 'OAK' 'SMF' 'OMA' 'TYS' 'SDF' 'PVD' 'PSE' 'SAV' 'BUR'
 'DSM' 'TUL' 'BHM' 'CAE' 'OKC' 'HDN' 'BZN' 'MTJ' 'EYW' 'PSP' 'ACK' 'ABQ'
 'ILM' 'BGR' 'MVY' 'SBN' 

### Explanation of the Code

This code aims to:
1. Build a **predictive model** for a binary classification task.
2. **Mitigate bias** related to a sensitive feature (`carrier` in this case) using Fairlearn's fairness reduction techniques.
3. Assess both the **fairness** and **performance** of the model.

### Detailed Steps

#### 1. **Data Loading and Inspection**
```python
data_path = './DataSets/KaggleData/flights_gold.csv'
df = pd.read_csv(data_path)
print("Available columns in dataset:", df.columns)
```
- The dataset is loaded from the specified path.
- The columns are printed to verify the structure of the data and ensure that all required columns are present.

#### 2. **Target Transformation**
```python
print("Unique target values in 'dest':", df['dest'].unique())
binary_mapping = {'EWR': 0, 'SFO': 1}
df['target_binary'] = df['dest'].map(binary_mapping)
```
- The target column (`dest`) represents flight destinations. It's converted into a binary variable:
  - `EWR` is mapped to `0`
  - `SFO` is mapped to `1`
- If any rows have destinations not mapped, those rows are dropped:
```python
if df['target_binary'].isnull().sum() > 0:
    df = df.dropna(subset=['target_binary'])
```

#### 3. **Feature Selection**
```python
X = df.drop(columns=['id', 'dest', 'target_binary'])
y = df['target_binary']
```
- The features (`X`) are created by dropping irrelevant or target-related columns (`id`, `dest`, `target_binary`).
- The target (`y`) is the binary-transformed column.

#### 4. **Data Splitting**
```python
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```
- The dataset is split into training (80%) and testing (20%) subsets.

#### 5. **Categorical Encoding**
```python
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)
```
- Categorical features in `X` are encoded into numerical values using one-hot encoding.
- One-hot encoding ensures that each category is represented as a binary column.

#### 6. **Bias Mitigation with Fairlearn**
```python
sensitive_feature = 'carrier'
X_train_encoded = pd.DataFrame(X_train_encoded)
X_train_encoded[sensitive_feature] = X_train[sensitive_feature].values
```
- `carrier` is identified as the **sensitive feature** (e.g., airline operator).
- This column is appended back to the encoded training data.

#### 7. **Training the Fairness-Aware Model**
```python
mitigator = ExponentiatedGradient(estimator=LinearRegression(),
                                  constraints=DemographicParity())
mitigator.fit(X_train_encoded, y_train, sensitive_features=X_train[sensitive_feature])
```
- An **Exponentiated Gradient** mitigator is used to enforce **Demographic Parity** while training a `LinearRegression` model:
  - **Demographic Parity** ensures that the prediction is independent of the sensitive feature, meaning the likelihood of predicting `1` is similar across different groups defined by `carrier`.

#### 8. **Predictions and Evaluation**
```python
y_pred_mitigated = mitigator.predict(X_test_encoded)
dp_diff = demographic_parity_difference(y_test, y_pred_mitigated, sensitive_features=X_test[sensitive_feature])
mae = mean_absolute_error(y_test, y_pred_mitigated)
```
- Predictions are made using the fairness-aware model.
- Two key metrics are calculated:
  1. **Demographic Parity Difference (`dp_diff`)**: Measures fairness by comparing the prediction rates across groups. A lower value indicates better fairness.
  2. **Mean Absolute Error (`mae`)**: Evaluates the model's prediction accuracy.

#### 9. **Results**
```python
print("Demographic Parity Difference after mitigation:", dp_diff)
print("Mean Absolute Error after mitigation:", mae)
```
- The results display:
  - How well the model performed on fairness (low `dp_diff` is better).
  - How accurate the model is (low `mae` is better).

---

### Goal of the Code
- **Bias Mitigation**: Train a model that reduces unfair advantages/disadvantages for groups defined by the sensitive feature (`carrier`).
- **Model Accuracy**: Balance fairness with prediction accuracy.
- **Fairness Metric**: Use **Demographic Parity Difference** to measure fairness improvement.

This pipeline is particularly useful for analyzing and mitigating bias in real-world datasets where fairness is a priority (e.g., hiring, lending, or resource allocation). Let me know if you'd like clarification on specific parts!